# <p style="text-align: center;"> FINAL PROJECT AI4AV RBE595 </p>
## <p style="text-align: center;"> Worcester Polytechnic Institute </p>
## <p style="text-align: center;"> April 2022 </p>

#### <p style="text-align: center;"> Aakash Rohra </p>
 
#### <p style="text-align: center;"> Devesh Datwani </p>

## <p style="text-align: center;"> Abstract </p> 

This project presents a method based on a convolution neural network to predict agent motion for autonomous driving. The method helps improve/accelerate the solving of self-driving cars by a data driven approach to predict agent motion around the ego vehicle. We train an architecture based on ResNet on Woven Planets Level 5 prediction dataset. The goal of the project is to achieve a high level of prediction accuracy in both homogeneous and heterogeneous urban driving conditions. We evaluate the model on a pre-training split test dataset. Further we investigate the computational efficiency of such a model on a mobile device. A robust architecture may help in ensuring enhanced safety in self-driving modes of autonomous cars as well as reducing road mishaps that occur due to driver error. 


<i style="font-size:20px;"> Importing necessary libraries </i>

In [57]:
from typing import Dict

from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet50
from tqdm import tqdm

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

import os

<i style="font-size:20px;"> Loading the dataset and rasterizing to Bird's Eye View </i>

In [76]:
zarr_dt = ChunkedDataset("sample.zarr")
zarr_dt.open()


cfg = load_config_data("agent_motion_config.yaml")
rast = build_rasterizer(cfg, LocalDataManager("sample.zarr"))
dataset = AgentDataset(cfg, zarr_dt, rast)

agent_idxs = range(0, len(dataset))
for agent_idx in tqdm(agent_idxs):
    data = dataset[agent_idx]
    img = data["image"]  # BEV input
    translations = data["target_positions"]

100%|██████████| 111634/111634 [29:52<00:00, 62.28it/s]


In [82]:
from IPython.display import display, clear_output
import PIL
 
cfg["raster_params"]["map_type"] = "py_semantic"
rast = build_rasterizer(cfg, dm)

zarr_dataset = ChunkedDataset("sample.zarr")
dataset = EgoDataset(cfg, zarr_dataset, rast)
scene_idx = 1
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"], data["raster_from_agent"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, TARGET_POINTS_COLOR, yaws=data["target_yaws"])
    clear_output(wait=True)
    display(PIL.Image.fromarray(im))

AssertionError: scene_idx 1 is over len 0

In [45]:
for frame in data.frames:
    print(frame.dtype)
    break

[('timestamp', '<i8'), ('agent_index_interval', '<i8', (2,)), ('traffic_light_faces_index_interval', '<i8', (2,)), ('ego_translation', '<f8', (3,)), ('ego_rotation', '<f8', (3, 3))]


In [47]:
for tl_face in data.tl_faces:
    print(tl_face.dtype)
    break

[('face_id', '<U16'), ('traffic_light_id', '<U16'), ('traffic_light_face_status', '<f4', (3,))]


In [51]:

cfg = load_config_data("CONFIG PATH")
rast = build_rasterizer(cfg, LocalDataManager("DATASET PATH"))
dataset = AgentDataset(cfg, zarr_dt, rast)

agent_idxs = range(0, len(dataset))
for agent_idx in tqdm(agent_idxs):
   data = dataset[agent_idx]
   img = data["image"]  # BEV input
   translations = data["target_positions"]  # future translations for the agent

FileNotFoundError: [Errno 2] No such file or directory: 'CONFIG PATH'

In [56]:
import os
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "."
dm = LocalDataManager(None)
# get config
cfg = load_config_data("./agent_motion_config.yaml")
print(cfg)

FileNotFoundError: [Errno 2] No such file or directory: './agent_motion_config.yaml'